# Imports

In [1]:
import pandas as pd
from tqdm import tqdm
import json
from datetime import datetime as dt
import ast

In [ ]:
import sys
ROOT = '../'
sys.path.append(ROOT)  # Add the root folder to the sys.path

# Import the modules
from config import *

# Reload the configuration
from importlib import reload
reload(sys.modules['config'])

# Import the reloaded modules
from config import *

# Retrieve Cointelegraph news

In [ ]:
import subprocess
import json

# Define the parameters
start = 0
MAX_LENGHT = 4000

# Parameters
short = "en"
slug = "bitcoin"
order = "postPublishedTime"
offset = str(start) # From the last news published
length = str(MAX_LENGHT) 

data = []
while True:
    try:
        print(f"Fetching data from {start} to {start + MAX_LENGHT}")
        # Define the curl command with parameters
        curl_command = [
            "curl",
            "https://conpletus.cointelegraph.com/v1/",
            "--compressed",
            "-X", "POST",
            "-H", "Accept-Encoding: gzip, deflate",
            "-H", "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0",
            "-H", "Accept: application/graphql-response+json, application/graphql+json, application/json, text/event-stream, multipart/mixed",
            "-H", "Accept-Language: en-US,en;q=0.5",
            "-H", "Accept-Encoding: gzip, deflate, br, zstd",
            "-H", "Referer: https://cointelegraph.com/",
            "-H", "baggage: ",
            "-H", "sentry-trace: ",
            "-H", "content-type: application/json",
            "-H", "Origin: https://cointelegraph.com",
            "-H", "Connection: keep-alive",
            "-H", "Sec-Fetch-Dest: empty",
            "-H", "Sec-Fetch-Mode: cors",
            "-H", "Sec-Fetch-Site: same-site",
            "-H", "Priority: u=4",
            "-H", "TE: trailers",
            "--data-raw", '{"operationName":"TagPageQuery","query":"query TagPageQuery($short: String, $slug: String!, $order: String, $offset: Int!, $length: Int!) {\\n  locale(short: $short) {\\n    tag(slug: $slug) {\\n      id\\n      slug\\n      avatar\\n      createdAt\\n      updatedAt\\n      redirectRelativeUrl\\n      alternates {\\n        short\\n        domain\\n        id\\n        code\\n        __typename\\n      }\\n      tagTranslates {\\n        id\\n        title\\n        metaTitle\\n        pageTitle\\n        description\\n        metaDescription\\n        keywords\\n        __typename\\n      }\\n      posts(order: $order, offset: $offset, length: $length) {\\n        data {\\n          id\\n          slug\\n          views\\n          postTranslate {\\n            id\\n            title\\n            avatar\\n            published\\n            publishedHumanFormat\\n            leadText\\n            author {\\n              id\\n              slug\\n              authorTranslates {\\n                id\\n                name\\n                __typename\\n              }\\n              __typename\\n            }\\n            __typename\\n          }\\n          category {\\n            id\\n            slug\\n            __typename\\n          }\\n          author {\\n            id\\n            slug\\n            authorTranslates {\\n              id\\n              name\\n              __typename\\n            }\\n            __typename\\n          }\\n          postBadge {\\n            id\\n            label\\n            postBadgeTranslates {\\n              id\\n              title\\n              __typename\\n            }\\n            __typename\\n          }\\n          showShares\\n          showStats\\n          __typename\\n        }\\n        postsCount\\n        __typename\\n      }\\n      __typename\\n    }\\n    __typename\\n  }\\n}","variables":{"cacheTimeInMS":300000,"length":'+length+',"offset":'+offset+',"order":"'+order+'","short":"'+short+'","slug":"'+slug+'"}}',
        ]

        # Execute the curl command
        result = subprocess.run(curl_command, capture_output=True, text=True)

        # Turn result.stdout into json
        data.append(json.loads(result.stdout))

        print(f"Data fetched from {start} to {start + MAX_LENGHT}")

        # Save the last date
        last_date = data[-1]['data']['locale']['tag']['posts']['data'][-1]['postTranslate']['published']

        # Convert into YYYY-MM-DD format
        last_date = dt.strptime(last_date, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d")

        print(f"last post published at {last_date}")

        # Check if the last date is before the start date
        if last_date < START_DATE:
            print(f"Reached the start date {START_DATE}")
            break 

        # Increase the offset
        start += MAX_LENGHT
        offset = str(start)
        
    except Exception as e:
        # Reached the end of the data
        print(e)
        break

In [ ]:
1=a

In [ ]:
# Save in a list all the news
news = []
for d in data:
    news.extend(d['data']['locale']['tag']['posts']['data'])

print(len(news))

print(news[0])

In [ ]:
import pandas as pd

# List to hold dictionaries
news_list = []

# Loop through news items and create dictionaries
for new in news:
    news_dict = {
        "id": new["id"],
        "slug": new["slug"],
        "views": new["views"],
        "title": new["postTranslate"]["title"],
        "published": new["postTranslate"]["published"],
        "leadtext": new["postTranslate"]["leadText"]
    }
    news_list.append(news_dict)

# Create DataFrame from list of dictionaries
df = pd.DataFrame(news_list)

# Display the first 5 rows of the DataFrame
df

In [ ]:
# Convert "2013-08-05T10:30:00+01:00" to "2024-07-17 15:30:28"
df['published'] = df['published'].apply(lambda x: dt.strptime(x, '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d %H:%M:%S'))

# Display the first 5 rows of the DataFrame
df

In [7]:
# Save the DataFrame to a CSV file
output_file = os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_raw.csv')
df.to_csv(output_file, index=False)

# [ADDON] Get "body" field of Coitelegraph news

In [8]:
# Load the DataFrame from the CSV file
input_file = os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_raw.csv')
df = pd.read_csv(input_file)

In [ ]:
# Add new column "body" to cointelegraph_with_body_with_sentiment between "leadtext" and "sentiment"
cointelegraph_with_body = df.copy()
cointelegraph_with_body.insert(6, "body", 'None')
cointelegraph_with_body

In [10]:
def find_text_to_remove(body):
    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Related:</em> inside them
    # Find all the <p> tags
    text_to_remove = []
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Related:"
                if "Related:" in em.get_text():
                    text_to_remove.append(p)
                    
    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Magazine:</em> inside them
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Magazine:"
                if "Magazine:" in em.get_text():
                    text_to_remove.append(p)

    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Recent:</em> inside them
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Recent:"
                if "Recent:" in em.get_text():
                    text_to_remove.append(p)

    return text_to_remove

In [11]:
import requests
from bs4 import BeautifulSoup

def retrieve_html(url):
    soup = None

    # With Charles Proxy/HTTP Toolkit
    headers = {
        "Host": "cointelegraph.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://cointelegraph.com/tags/bitcoin",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Priority": "u=0, i",
        "TE": "trailers"
    }

    # # Retrieve the cookie from the secrets/cookies.json file
    # with open(os.path.join(ROOT, 'secrets/cookies.json')) as file:
    #     cookie = json.load(file)['cookie']

    # # With Requestly Proxy
    # headers = {
    #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
    #     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
    #     "Accept-Language": "en-US,en;q=0.5",
    #     "Accept-Encoding": "gzip, deflate, br, zstd",
    #     "Connection": "keep-alive",
    #     "Cookie": cookie,
    #     "Upgrade-Insecure-Requests": "1",
    #     "Sec-Fetch-Dest": "document",
    #     "Sec-Fetch-Mode": "navigate",
    #     "Sec-Fetch-Site": "none",
    #     "Sec-Fetch-User": "?1",
    #     "Priority": "u=0, i"
    # }

    # Get the body of the article from the URL
    response = requests.get(url, headers=headers)
      
    # Retry if the status code is not 200
    if response.status_code != 200:
        count = 0
        while response.status_code != 200 and count < 5:
            response = requests.get(url, headers=headers)
            count += 1

    else:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

    return soup

In [12]:
def get_cointelegraph_body(url, cus_class):
    import requests

    try:
        # Get the body of the article from the URL
        soup = retrieve_html(url)

        if soup == None:
            print(f"Failed to get the content of the article: {url}")
            return 'None'

        # Find the body of the article
        body = soup.find('div', class_=cus_class)

        # Clean the body
        text_to_remove = find_text_to_remove(body)
    except Exception as e:
        print(e)
        print(f"Failed to find the body of the article: {url}")
        return 'None'

    # Get the text of the body
    body_text = body.get_text()

    # Remove the text that contains "Related:"
    for text in text_to_remove:
        body_text = body_text.replace(text.get_text(), '')
    
    if body_text == '':      
        print(f"Failed to find the body of the article in the URL: {url}")
        return 'None'
    return body_text

In [ ]:
from tqdm import tqdm
from time import sleep
import os

# For each news open cointelegrap.com/news/{slug} and save the content
for index, row in tqdm(cointelegraph_with_body.iterrows(), total=len(cointelegraph_with_body)):
    try:
        # Get the slug
        slug = row['slug']

        # Open the URL
        url = f"https://cointelegraph.com/news/{slug}"
        
        # Get the body
        cus_class = 'post-content relative'
        body = get_cointelegraph_body(url, cus_class)

        # Save the body into the dataset
        cointelegraph_with_body.at[index, 'body'] = body
    except Exception as e:
       print(e)
       cointelegraph_with_body.at[index, 'body'] = 'None'

In [ ]:
cointelegraph_with_body

In [ ]:
# Save the dataset to a CSV file
output_file = os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_with_body.csv')
cointelegraph_with_body.to_csv(output_file, index=False)

# [ADDON] Retrieve 'None' body

In [ ]:
# Load the dataset
cointelegraph_with_body = pd.read_csv(os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_with_body.csv'))

In [ ]:
# Check for None values
cointelegraph_with_null_body = cointelegraph_with_body[cointelegraph_with_body['body'].isnull()]
cointelegraph_with_null_body

In [ ]:
def find_text_to_remove(body):
    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Related:</em> inside them
    # Find all the <p> tags
    text_to_remove = []
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Related:"
                if "Related:" in em.get_text():
                    text_to_remove.append(p)
                    
    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Magazine:</em> inside them
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Magazine:"
                if "Magazine:" in em.get_text():
                    text_to_remove.append(p)

    # Extract all the <p ...>...</p> tags that contains <strong>...</strong> with <em>Recent:</em> inside them
    p_tags = body.find_all('p')
    for p in p_tags:
        # Find all the <strong> tags inside the <p> tag
        strong_tags = p.find_all('strong')
        for strong in strong_tags:
            # Find all the <em> tags inside the <strong> tag
            em_tags = strong.find_all('em')
            for em in em_tags:
                # Check if the <em> tag contains the text "Recent:"
                if "Recent:" in em.get_text():
                    text_to_remove.append(p)

    return text_to_remove

In [ ]:
def get_cointelegraph_body(url, cus_class):
    import requests
    from bs4 import BeautifulSoup

    # Get the body of the article from the URL
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0'
    }
    response = requests.get(url, headers=headers)
    
    # Retry if the status code is not 200
    if response.status_code != 200:
        count = 0
        while response.status_code != 200 and count < 10:
            response = requests.get(url, headers=headers)
            count += 1

    if response.status_code != 200:
        print(f"Failed to get the URL: {url}")
        return 'None'
        
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the body of the article
    body = soup.find('div', class_=cus_class)

    try:
        # Clean the body
        text_to_remove = find_text_to_remove(body)
    except Exception as e:
        print(e)
        print(f"Failed to find the body of the article in the URL: {url}")
        return 'None'

    # Get the text of the body
    body_text = body.get_text()

    # Remove the text that contains "Related:"
    for text in text_to_remove:
        body_text = body_text.replace(text.get_text(), '')
    
    if body_text == '':      
        print(f"Failed to find the body of the article in the URL: {url}")

        return 'None'
    return body_text

In [ ]:
# For each news open cointelegrap.com/news/{slug} and save the content
for index, row in tqdm(cointelegraph_with_null_body.iterrows(), total=len(cointelegraph_with_null_body)):
    try:
        # Get the slug
        slug = row['slug']

        # Open the URL
        url = f'https://www.cointelegraph.com/news/{slug}' # 'https://www.cointelegraph.com/news/{slug}' | 'https://www.cointelegraph.com/explained/{slug}'
        
        # Get the body
        cus_class = 'post-content relative shadow-[0_2px_32px_0px_#00000014] bg-custom-gray-50' # 'post-content relative' | 'post-content relative shadow-[0_2px_32px_0px_#00000014] bg-custom-gray-50'
        body = get_cointelegraph_body(url, cus_class)

        # Save the body into the dataset
        cointelegraph_with_null_body.at[index, 'body'] = body
    except Exception as e:
        print(e)
        # Set the body to None
        cointelegraph_with_null_body.at[index, 'body'] = 'None'

    # Save the dataset to a CSV file
    output_file = os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_with_body.csv')

In [ ]:
# Merge cointelegraph_with_null_body with cointelegraph_with_body
cointelegraph_with_body.update(cointelegraph_with_null_body)

In [ ]:
# Check for None values
cointelegraph_with_null_body = cointelegraph_with_null_body[cointelegraph_with_null_body['body'].isnull()]
cointelegraph_with_null_body

In [ ]:
# Remove the remaining rows with body equal to None
cointelegraph_with_body.dropna(inplace=True)

In [ ]:
cointelegraph_with_body

In [ ]:
# Save the dataset to a CSV file
output_file = os.path.join(ROOT, NEWS_DATASET_PATH, 'cointelegraph_with_body.csv')
cointelegraph_with_body.to_csv(output_file, index=False)

# Generate daily and hourly Cointelegraph dataset

In [ ]:
# Open cointelegraph dataset
cointelegraph_with_body = pd.read_csv(os.path.join(ROOT, NEWS_DATASET_PATH, "cointelegraph_with_body.csv"))
cointelegraph_with_body

In [8]:
# Generate two temp dataset (daily and hourly) with just the timestamp column that start from START_DATE and end at END_DATE
# Daily dataset has the format: 2024-03-16
# Hourly dataset has the format: 2024-03-16 20:00:00

# Define the start date
start_date = cointelegraph_with_body['published'].min() # Default: START_DATE
end_date = cointelegraph_with_body['published'].max() # Default: END_DATE

cointelegraph_daily = pd.date_range(start=start_date, end=end_date, freq='D').to_frame(index=False, name='timestamp')
cointelegraph_daily['timestamp'] = cointelegraph_daily['timestamp'].dt.date

# start_date = cointelegraph_with_body['published'].min()+" 00:00:00" # Default: START_DATE

# For the hourly dataset, generate 2 coluns: timestamp_begin and timestamp_end where timestamp_end is timestamp_begin + 1 hour
# cointelegraph_hourly = pd.date_range(start=start_date, end=END_DATE, freq='h').to_frame(index=False, name='timestamp')
# cointelegraph_hourly['timestamp_begin'] = cointelegraph_hourly['timestamp']
# cointelegraph_hourly['timestamp_end'] = cointelegraph_hourly['timestamp'] + pd.Timedelta(hours=1)
# cointelegraph_hourly.drop(columns=['timestamp'], inplace=True)

# Turn the timestamp column into string
cointelegraph_daily['timestamp'] = cointelegraph_daily['timestamp'].astype(str)
# cointelegraph_hourly['timestamp_begin'] = cointelegraph_hourly['timestamp_begin'].astype(str)
# cointelegraph_hourly['timestamp_end'] = cointelegraph_hourly['timestamp_end'].astype(str)

# Add 'cointelegraph' column
cointelegraph_daily['cointelegraph'] = None
# cointelegraph_hourly['cointelegraph'] = None

In [ ]:
cointelegraph_daily

In [10]:
# cointelegraph_hourly

In [ ]:
# Generate cointelegraph_daily
cointelegraph_daily_copy = cointelegraph_daily.copy()

# Iterate over the daily dataset
for index, row in tqdm(cointelegraph_daily_copy.iterrows(), total=len(cointelegraph_daily_copy)):
    # Get the timestamp
    curr_timestamp = row['timestamp']
    # Select the cointelegraph rows items that have been published during the curr_timestamp, save them as a list
    # Example: if curr_timestamp is 2018-01-01, then select the news items that have been published from 2018-01-01 00:00:00 to 2018-01-01 23:59:59, news = [news1, news2, news3]
    filtered_news = cointelegraph_with_body[(cointelegraph_with_body['published'].str.contains(curr_timestamp))]

    if len(filtered_news) == 0:
        cointelegraph_daily_copy.at[index, 'cointelegraph'] = str([])
    else:
        # Convert news dataframe to a list
        filtered_news = str(filtered_news.values.tolist())
        # Append the news list to the news column
        cointelegraph_daily_copy.at[index, 'cointelegraph'] = filtered_news
cointelegraph_daily_copy

In [ ]:
# Count the number of NOT empty rows (different from []) in the cointelegraph column
not_empty_rows = cointelegraph_daily_copy[cointelegraph_daily_copy['cointelegraph'] != '[]']
print("Total number of NOT '[]' occurrences in the cointelegraph column:", not_empty_rows.shape[0])
not_empty_rows

In [ ]:
# Show an example
row = not_empty_rows['cointelegraph'][0]
print(f"Row has {len(row)} news")
print(f"List of news: {row}")
row

In [ ]:
# Count the number of empty rows (equal to '[]') in the cointelegraph column
empty_rows = cointelegraph_daily_copy[cointelegraph_daily_copy['cointelegraph'] == '[]']
print("Total number of '[]' occurrences in the cointelegraph column:", empty_rows.shape[0])
empty_rows

In [ ]:
# Set the timestamp as the index
cointelegraph_daily_copy = cointelegraph_daily_copy.set_index('timestamp', drop=False)
cointelegraph_daily = cointelegraph_daily_copy
cointelegraph_daily

In [16]:
# import matplotlib.pyplot as plt

# # Increase the figure size
# plt.figure(figsize=(24, 8))

# # Plot the number of news items per day
# plt.plot(cointelegraph_daily['timestamp'], cointelegraph_daily['cointelegraph'].str.count('\[[^\]]'))

# # Set x intervals
# plt.xticks(cointelegraph_daily['timestamp'][::60], rotation=45)

# # Show the plot
# plt.show()

In [17]:
# # Generate cointelegraph_hourly
# cointelegraph_hourly_copy = cointelegraph_hourly.copy()

# # Iterate over the hourly dataset
# for index, row in tqdm(cointelegraph_hourly_copy.iterrows(), total=len(cointelegraph_hourly_copy)):
#     # Get the timestamp
#     timestamp_begin = row['timestamp_begin']
#     timestamp_end = row['timestamp_end']
#     # Select the cointelegraph rows items that have been published between timestamp_begin and timestamp_end, save them as a list
#     # Example: if timestamp_begin is 2018-01-01 00:00:00 and timestamp_end is 2018-01-01 01:00:00
#     # select the news items that have been published from 2018-01-01 00:00:00 to 2018-01-01 00:59:59
#     # news = [news1, news2, news3]
#     filtered_news = cointelegraph_with_body[
#         (cointelegraph_with_body['published'] >= timestamp_begin) & 
#         (cointelegraph_with_body['published'] < timestamp_end)
#     ]

#     if len(filtered_news) == 0:
#         cointelegraph_hourly_copy.at[index, 'cointelegraph'] = str([])
#     else:
#         # Convert news dataframe to a list
#         filtered_news = str(filtered_news.values.tolist())
#         # Append the news list to the news column
#         cointelegraph_hourly_copy.at[index, 'cointelegraph'] = filtered_news
# cointelegraph_hourly_copy

In [18]:
# # Count the number of NOT empty rows (different from []) in the cointelegraph column
# not_empty_rows = cointelegraph_hourly_copy[cointelegraph_hourly_copy['cointelegraph'] != '[]']
# print("Total number of NOT '[]' occurrences in the cointelegraph column:", not_empty_rows.shape[0])
# not_empty_rows

In [19]:
# # Show an example
# row = not_empty_rows['cointelegraph'][45230]
# print(f"Row has {len(row)} news")
# print(f"List of news: {row}")
# row

In [20]:
# # Count the number of empty rows (equal to '[]') in the cointelegraph column
# empty_rows = cointelegraph_hourly_copy[cointelegraph_hourly_copy['cointelegraph'] == '[]']
# print("Total number of '[]' occurrences in the cointelegraph column:", empty_rows.shape[0])
# empty_rows

In [21]:
# # Set the timestamp as the index
# cointelegraph_hourly_copy = cointelegraph_hourly_copy.set_index('timestamp_begin', drop=False)
# cointelegraph_hourly = cointelegraph_hourly_copy
# cointelegraph_hourly

In [22]:
# import matplotlib.pyplot as plt

# # Increase the figure size
# plt.figure(figsize=(24, 8))

# # Plot the number of news items per day
# plt.plot(cointelegraph_hourly['timestamp'], cointelegraph_hourly['cointelegraph'].str.count('\[[^\]]'))

# # Group x axis by days
# plt.xticks(cointelegraph_hourly['timestamp'][::24*60], rotation=45)


# # Show the plot
# plt.show()

In [23]:
# Save the datasets
cointelegraph_daily.to_csv(os.path.join(ROOT, NEWS_DATASET_PATH, "cointelegraph_daily_grouped.csv"), index=False)

# # Save the datasets
# cointelegraph_hourly.to_csv(os.path.join(ROOT, NEWS_DATASET_PATH, "cointelegraph_hourly_grouped.csv"), index=False)